## Grouping Home Health Care and Hopital Readmission Data by Zip Code
***
My initial EDA determined that a lack of observations may have lead to unexpected or inaccurate results.  I will now be joining the data on zip codes rather than on states to generate more observations.

In [34]:
# Importing Necessary Tools
import pandas as pd
import numpy as np

In [35]:
#Pull File Into Database and Set Column Names
col = ['hospital_name', 'provider_number', 'state', 'measure', 'discharges','footnote',
           'readmission_ratio','predicted_rate','expected_rate','readmissions','starte_date','end_Date']
df = pd.read_csv('Readmissions.csv')
df.columns=col

**Initial removal of the following columns:**<br>
-  Measure: The observations are to be grouped together by zip code to get the overall state readmission ratio.  Subsequently, rendering this column unneccessary to this analysis.
-  Footnote:  Footnotes are associated with a lack of information.  Most of which will be removed in the cleaning process.
-  Start_Date:  Does not provide any useful information for this analysis.  Also the same for all rows.
-  End_Date: Does not provide any useful information fro this analysis.  Also the same for all rows.
-  Discharges:  Currently not needed for this analysis.  High number of null values (8,072).
-  Readmissions:  Currently not needed for this analysis.  High number of null values (8,192).
-  Provider Number:  Not a useful identifier as a hospital identifier is not associated with a HHC identifier.

In [36]:
# Removing Columns

usecols= ['hospital_name','state','readmission_ratio',
           'predicted_rate','expected_rate']
df=df[usecols]
df.head()

,hospital_name,state,readmission_ratio,predicted_rate,expected_rate
0,HIGHLANDS MEDICAL CENTER,AL,Not Available,Not Available,Not Available
1,CLAY COUNTY HOSPITAL,AL,0.9853,14.4,14.6
2,NORTHEAST ALABAMA REGIONAL MEDICAL CENTER,AL,1.4044,6.1,4.3
3,NORTHEAST ALABAMA REGIONAL MEDICAL CENTER,AL,0.9653,16.7,17.3
4,ATHENS LIMESTONE HOSPITAL,AL,1.0204,4.3,4.2


In [37]:
# Coerce Discharges, Readmission Ratios, Predicted Rates, Expected Rates, and Readmissions to get NaNs
tonumeric=['readmission_ratio','predicted_rate','expected_rate',]
dfa = df[tonumeric].apply(pd.to_numeric, errors='coerce')
#Setting up additional columns to concatinate
dfb = df[['hospital_name','state']]

In [38]:
# Concatenating Data Back Together and Confirming DataFrame Integrity
df2= pd.concat([dfb,dfa], axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19830 entries, 0 to 19829
Data columns (total 5 columns):
hospital_name        19830 non-null object
state                19830 non-null object
readmission_ratio    14411 non-null float64
predicted_rate       14411 non-null float64
expected_rate        14411 non-null float64
dtypes: float64(3), object(2)
memory usage: 774.7+ KB


***
Now it is time to read in the zip code data and attach it to each facility.  Zip code data retrieved at CMS's website [Direct Link Here](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Provider-of-Services/POS2016.html).

In [39]:
# Read Data and assign column names
df = pd.read_csv('zipcodes.csv')
col = ['name', 'state', 'zip']
df.columns=col

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469551 entries, 0 to 469550
Data columns (total 3 columns):
name     469551 non-null object
state    469551 non-null object
zip      469551 non-null int64
dtypes: int64(1), object(2)
memory usage: 10.7+ MB


In [40]:
# Merge Data on Name and State to Add Zip a Total Loss of 3,377 Observations Due to No Zip Found for Hospital.
final_hospital = pd.merge(df2,df, left_on=['hospital_name', 'state'], right_on=['name','state'])
# Drop Redundent Name Column
final_hospital = final_hospital.drop('name', axis=1)

In [41]:
# Save and Print Final DataFrame Heading
final_hospital.to_csv('Readmissions_2.csv')
final_hospital.head()

,hospital_name,state,readmission_ratio,predicted_rate,expected_rate,zip
0,HIGHLANDS MEDICAL CENTER,AL,NaN,NaN,NaN,35768
1,HIGHLANDS MEDICAL CENTER,AL,NaN,NaN,NaN,35768
2,HIGHLANDS MEDICAL CENTER,AL,0.9724,15.6,16.1,35768
3,HIGHLANDS MEDICAL CENTER,AL,0.9724,15.6,16.1,35768
4,HIGHLANDS MEDICAL CENTER,AL,0.9872,17.9,18.2,35768


***
Now Let's Bring In and Clean the Expanded Home Health Care Agencies Spreadsheet


In [73]:
df = pd.read_csv('HHC_Agencies.csv')
# Creating List and Renaming Columns
col = ['state', 'cms_number','name','address','city','zip','phone','own_type','nursing_care',
       'physical_therapy', 'occupational_therapy','pathology_services','medical_soc_services',
       'home_health_aid','cert_date','star_rating','footnote','timeliness','footnote','rx_ed',
       'footnote','fall_risk','footnote','depression_check','footnote','flu_shot','footnote',
       'pneumonia_shot','footnote','d_foot_care','footnote','move_buff','footnote','in_out_bed_buff',
       'footnote','bathing_buff','footnote','move_pain_debuff','footnote','breathing_buff','footnote',
       'healing_buff','footnote','oral_rx_buff','footnote','hospital_admit','footnote','urgent_noadmit',
      'footnote','readmit_expectation','footnote','er_admit_expectation','footnote','footnote']
df.columns=col

In [74]:
df.head()

,state,cms_number,name,address,city,zip,phone,own_type,nursing_care,physical_therapy,...,footnote,hospital_admit,footnote,urgent_noadmit,footnote,readmit_expectation,footnote,er_admit_expectation,footnote,footnote
0,AL,17000,ALABAMA DEPARTMENT OF PUBLIC HEALTH HOME CARE,"201 MONROE STREET, THE RSA TOWER, SUITE 1200",MONTGOMERY,36104,3342065341,Government - State/ County,True,True,...,This measure currently does not have data or p...,NaN,This measure currently does not have data or p...,NaN,This measure currently does not have data or p...,Not Available,This measure currently does not have data or p...,Not Available,This measure currently does not have data or p...,
1,AL,17008,JEFFERSON COUNTY HOME CARE,2201 ARLINGTON AVENUE,BESSEMER,35020,2059169500,Government - State/ County,True,True,...,NaN,NaN,The number of patient episodes for this measur...,NaN,The number of patient episodes for this measur...,Not Available,The number of patient episodes for this measur...,Not Available,The number of patient episodes for this measur...,
2,AL,17009,ALACARE HOME HEALTH & HOSPICE,2970 LORNA ROAD,BIRMINGHAM,35216,2058242680,Proprietary,True,True,...,NaN,18.3,NaN,11.4,NaN,Worse Than Expected,NaN,Worse Than Expected,NaN,
3,AL,17013,KINDRED AT HOME,1239 RUCKER BLVD,ENTERPRISE,36330,3343470234,Proprietary,True,True,...,NaN,15.5,NaN,15.1,NaN,Same As Expected,NaN,Worse Than Expected,NaN,
4,AL,17014,AMEDISYS HOME HEALTH,68278 MAIN STREET,BLOUNTSVILLE,35031,8664864919,Proprietary,True,True,...,NaN,18.9,NaN,12.1,NaN,Same As Expected,NaN,Same As Expected,NaN,


**Removal of the following columns:**<br>
-  All Footnotes:  Footnotes are mostly empty and associated with a lack of information.  Most empty data will be removed in the cleaning process.
-  Address:  Does not provide any useful information for this analysis.
-  City:  Does not provide any useful information for this analysis.
-  CMS Number:  Does not provide any useful information for this analysis.
-  Phone:  Does not provide any useful information for this analysis.
-  Type of Ownership:  Does not provide any useful information for this analysis.
-  Date Certified: Does not provide any useful information fro this analysis.  
-  Wound Care Improvement: Significant loss of data (7,400 null values).

In [75]:
# Removing Columns
usecols= ['state', 'name','zip','nursing_care','physical_therapy','occupational_therapy',
          'pathology_services','medical_soc_services','home_health_aid','star_rating','timeliness',
          'rx_ed','fall_risk','depression_check','flu_shot','pneumonia_shot','d_foot_care',
          'move_buff','in_out_bed_buff','bathing_buff','move_pain_debuff','breathing_buff',
          'oral_rx_buff','hospital_admit','urgent_noadmit','readmit_expectation',
          'er_admit_expectation']
df=df[usecols]
df.head()

,state,name,zip,nursing_care,physical_therapy,occupational_therapy,pathology_services,medical_soc_services,home_health_aid,star_rating,...,move_buff,in_out_bed_buff,bathing_buff,move_pain_debuff,breathing_buff,oral_rx_buff,hospital_admit,urgent_noadmit,readmit_expectation,er_admit_expectation
0,AL,ALABAMA DEPARTMENT OF PUBLIC HEALTH HOME CARE,36104,True,True,True,True,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Available,Not Available
1,AL,JEFFERSON COUNTY HOME CARE,35020,True,True,True,True,True,True,3.0,...,71.2,64.3,64.1,80.4,76.7,49.2,NaN,NaN,Not Available,Not Available
2,AL,ALACARE HOME HEALTH & HOSPICE,35216,True,True,True,True,True,True,4.0,...,79.4,75.4,83.5,85.9,81.3,72.4,18.3,11.4,Worse Than Expected,Worse Than Expected
3,AL,KINDRED AT HOME,36330,True,True,True,False,False,True,4.0,...,77.6,71.4,80.3,83.6,79.3,59.9,15.5,15.1,Same As Expected,Worse Than Expected
4,AL,AMEDISYS HOME HEALTH,35031,True,True,True,True,True,True,4.0,...,81.3,72.8,82.1,78.0,85.7,68.5,18.9,12.1,Same As Expected,Same As Expected


In [76]:
# Reviewing Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11678 entries, 0 to 11677
Data columns (total 27 columns):
state                   11678 non-null object
name                    11678 non-null object
zip                     11678 non-null int64
nursing_care            11678 non-null bool
physical_therapy        11678 non-null bool
occupational_therapy    11678 non-null bool
pathology_services      11678 non-null bool
medical_soc_services    11678 non-null bool
home_health_aid         11678 non-null bool
star_rating             8890 non-null float64
timeliness              9606 non-null float64
rx_ed                   9579 non-null float64
fall_risk               9380 non-null float64
depression_check        9582 non-null float64
flu_shot                9276 non-null float64
pneumonia_shot          9540 non-null float64
d_foot_care             8473 non-null float64
move_buff               8844 non-null float64
in_out_bed_buff         8749 non-null float64
bathing_buff            8887 n

In [77]:
# Reviewing Distinct Values for Readmission Expectations and ER Admission Expectations
df.readmit_expectation.unique()

array(['Not Available', 'Worse Than Expected', 'Same As Expected',
       'Better Than Expected'], dtype=object)

In [78]:
df.er_admit_expectation.unique()

array(['Not Available', 'Worse Than Expected', 'Same As Expected',
       'Better Than Expected'], dtype=object)

In [79]:
# Changing Categories to Numeric Values for Readmission Expectations and ER Admission Expectations
for n in range(len(df)):
    for m in range(25,27):
        if df.iloc[n,m] == 'Not Available':
            df.iloc[n,m] = 0
        elif df.iloc[n,m] == 'Worse Than Expected':
            df.iloc[n,m] = 1
        elif df.iloc[n,m] == 'Same As Expected':
            df.iloc[n,m] = 2
        elif df.iloc[n,m] == 'Better Than Expected':
            df.iloc[n,m] = 3
    

In [80]:
# Removing US Territories That Do Not Have Hospital Readmission Data
territories = ['PR','GU','MP','VI']
for i in range(len(territories)):
    final_hhc = df[df.state != territories[i]]


In [83]:
# The HHC Categories Suffer From Some Extreme Outliers as Indicated by Example Below:
final_hhc[final_hhc.flu_shot==.5]

,state,name,zip,nursing_care,physical_therapy,occupational_therapy,pathology_services,medical_soc_services,home_health_aid,star_rating,...,move_buff,in_out_bed_buff,bathing_buff,move_pain_debuff,breathing_buff,oral_rx_buff,hospital_admit,urgent_noadmit,readmit_expectation,er_admit_expectation
10435,TX,ST FRANCIS HEALTH CARE SERVICES INC,77036,True,True,True,True,True,True,2.0,...,49.2,34.9,84.2,33.1,69.5,68.5,NaN,NaN,2,2


These extreme outliers are likely mistakes in data entry as they do not make sense with the rest of the data.  As such, we must remove these outliers to protect the integrity of the grouping and the data overall.

In [84]:
# Removing Extreme Outliers from Categorical Data
columns = ['timeliness','rx_ed','fall_risk','depression_check','flu_shot','pneumonia_shot','d_foot_care','move_buff',
      'in_out_bed_buff','bathing_buff','move_pain_debuff','breathing_buff','oral_rx_buff','hospital_admit','urgent_noadmit']

# Initiate Starting Column Location
current_col = 10

# Loop through Data and Remove Exteme Outliers
for col in columns:
# Grab Mean and Standard Deviation from the Data
    u= np.mean(final_hhc[col])
    s= np.std(final_hhc[col])
# Iterate Through Data and Filter out Price Outliers by Marking Them as Null
    for n in range(len(final_hhc[col])):
        if final_hhc.iloc[n][current_col] < u-3*s or final_hhc.iloc[n][current_col] > u+3*s:
            final_hhc.set_value(n,col,None) 
    current_col +=1

In [86]:
# Save and Print Final DataFrame Heading
final_hhc.to_csv('HHC_Agencies_Cleaned.csv')
final_hhc.head()

,state,name,zip,nursing_care,physical_therapy,occupational_therapy,pathology_services,medical_soc_services,home_health_aid,star_rating,...,move_buff,in_out_bed_buff,bathing_buff,move_pain_debuff,breathing_buff,oral_rx_buff,hospital_admit,urgent_noadmit,readmit_expectation,er_admit_expectation
0,AL,ALABAMA DEPARTMENT OF PUBLIC HEALTH HOME CARE,36104,True,True,True,True,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,AL,JEFFERSON COUNTY HOME CARE,35020,True,True,True,True,True,True,3.0,...,71.2,64.3,64.1,80.4,76.7,49.2,NaN,NaN,0,0
2,AL,ALACARE HOME HEALTH & HOSPICE,35216,True,True,True,True,True,True,4.0,...,79.4,75.4,83.5,85.9,81.3,72.4,18.3,11.4,1,1
3,AL,KINDRED AT HOME,36330,True,True,True,False,False,True,4.0,...,77.6,71.4,80.3,83.6,79.3,59.9,15.5,15.1,2,1
4,AL,AMEDISYS HOME HEALTH,35031,True,True,True,True,True,True,4.0,...,81.3,72.8,82.1,78.0,85.7,68.5,18.9,12.1,2,2


***
Finaly it's time to build our Zipcode Based DataFrame!

In [87]:
# The Excessive Readmission Rate for Each Zip Code Could be Interesting.  Let's Build that Array First.

# Build Dictionary of States and Assign them to 0
dictionary ={}
for n in final_hospital.zip:
    if n in dictionary.keys():
        continue
    else:
        dictionary[n]=0
# Count the Number of Excessive Readmissions Per Zip Code
for x in range(len(final_hospital)): 
    if final_hospital.iloc[x][2]>1:
        dictionary[final_hospital.iloc[x][5]] +=1

In [88]:
# Convert Dictionary to DataFrame to Merge into the Final DataFrame
e_count = pd.DataFrame(list(dictionary.items()))
e_count.columns=['zip','excessive_count']

In [89]:
# Initializing and Creating New Dataframe To Group By Zip Code
states_zip= pd.DataFrame(final_hospital[['zip','state']])
states_zip= states_zip.drop_duplicates()
ffinal= pd.DataFrame(states_zip)
ffinal.columns=['zip','state']
ffinal = ffinal.reset_index()
# Merge Excessive Count DataFrame
ffinal = pd.merge(ffinal,e_count, on='zip')


In [90]:
# Group the Remaining Hospital Readmission Information into the Final DataFrame and Drop Old Index
ffinal['readmission_ratio'] = np.array(final_hospital.groupby('zip').readmission_ratio.mean())
ffinal['predicted_rate']= np.array(final_hospital.groupby('zip').predicted_rate.mean())
ffinal['expected_rate'] = np.array(final_hospital.groupby('zip').expected_rate.mean())
ffinal = ffinal.drop('index', axis=1)

Our Final DataFrame is Prepared!  Now it's time to prep the HHC Data for Merger!

In [91]:
# Convert Booleans to Binary
for n in range(3,9):
    final_hhc.iloc[:,n] = final_hhc.iloc[:,n].astype(int)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [92]:
# Initialize Merger DataFrame with the unique zip codes
gmerger = pd.DataFrame(final_hhc.zip)
gmerger = gmerger.drop_duplicates()
gmerger = gmerger.reset_index()
gmerger = gmerger.drop('index', axis=1)

In [93]:
#  Loop Through Columns Intended to be Grouped by Zip to get Grouped Mean
mean_cols = ['nursing_care','physical_therapy','occupational_therapy','pathology_services','medical_soc_services',
               'home_health_aid','star_rating','timeliness','rx_ed','fall_risk','depression_check','flu_shot','pneumonia_shot',
               'd_foot_care','move_buff','in_out_bed_buff','bathing_buff','move_pain_debuff','breathing_buff','oral_rx_buff',
               'hospital_admit','urgent_noadmit']

for col in mean_cols:
    gmerger[col] = np.array(final_hhc.groupby('zip')[col].mean())    

In [94]:
#  Loop Through Columns Intended to be Grouped by Zip to get Grouped Mode
mode_cols = ['readmit_expectation', 'er_admit_expectation']
from scipy.stats import mode

# Mode was Chosen as These are Categorical Indicators
for col in mode_cols:
    final_hhc[col] = final_hhc[col].astype(int)
    gmerger[col] = np.array(final_hhc.groupby('zip')[col].agg(lambda x: x.value_counts().index[0]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [95]:
gmerger.head()

,zip,nursing_care,physical_therapy,occupational_therapy,pathology_services,medical_soc_services,home_health_aid,star_rating,timeliness,rx_ed,...,move_buff,in_out_bed_buff,bathing_buff,move_pain_debuff,breathing_buff,oral_rx_buff,hospital_admit,urgent_noadmit,readmit_expectation,er_admit_expectation
0,36104,1,1.0,1.0,1.0,1.0,1.0,3.5,87.0,99.5,...,81.4,73.7,80.1,93.9,88.6,70.9,19.7,12.9,2,2
1,35020,1,1.0,1.0,0.0,1.0,1.0,3.5,92.8,96.6,...,77.3,73.6,80.4,89.1,83.1,62.1,NaN,9.0,2,2
2,35216,1,1.0,1.0,1.0,1.0,0.5,4.0,95.5,99.5,...,76.2,66.4,76.0,93.1,93.4,61.2,10.4,13.7,2,2
3,36330,1,1.0,1.0,1.0,1.0,1.0,3.5,92.0,95.3,...,73.2,73.0,78.9,86.2,91.9,60.7,18.5,3.0,2,2
4,35031,1,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


***
Time to put everything together!!

In [96]:
ffinal = pd.merge(ffinal,gmerger, on='zip')

In [97]:
ffinal.to_csv('final_by_zip.csv')
ffinal.tail()

,zip,state,excessive_count,readmission_ratio,predicted_rate,expected_rate,nursing_care,physical_therapy,occupational_therapy,pathology_services,...,move_buff,in_out_bed_buff,bathing_buff,move_pain_debuff,breathing_buff,oral_rx_buff,hospital_admit,urgent_noadmit,readmit_expectation,er_admit_expectation
1006,95901,CA,4,1.015283,15.016667,15.05,1,1.0,1.0,1.0,...,74.5,71.95,80.00,84.95,78.70,66.55,15.1,12.7,2,1
1007,86326,AZ,2,0.924010,13.650000,14.53,1,1.0,1.0,1.0,...,70.1,78.70,61.50,80.40,77.10,42.30,15.8,15.2,2,2
1008,55057,MN,2,1.009360,14.180000,14.30,1,1.0,1.0,1.0,...,60.1,82.50,76.45,59.70,75.55,74.50,22.5,15.0,2,2
1009,50112,IA,4,1.102800,15.200000,14.40,1,1.0,1.0,1.0,...,65.4,61.60,71.20,79.10,77.20,62.00,11.1,14.6,2,2
1010,52342,IA,2,0.966000,12.300000,12.45,1,1.0,0.0,1.0,...,70.3,71.90,67.50,87.00,71.60,63.50,16.6,15.0,1,1
